## Model 1 Just as it was


### Summary


In [1]:
import os
import re
import pandas as pd
import numpy as np
np.warnings.filterwarnings('ignore')

TRY = '2'
RANDOM_STATE = 42

DATADIR = './16kmono_UrbanSound8K'
OUTDIR = './result%s' % TRY # just a random name

labels = ['air_conditioner',  'car_horn', 'children_playing', 'dog_bark', 'drilling',
    'engine_idling', 'gun_shot','jackhammer', 'siren', 'street_music']
id2name = {i: name for i, name in enumerate(labels)}
name2id = {name: i for i, name in id2name.items()}
sample_rate = 16000

def load_data():
    """ Return 2 lists of indexes:
    [train_data_index, ...] for train
    [valid_data_index, ...] for validation
    """
    X_train = np.load('X_train.npy')
    y_train = np.load('y_train.npy')
    X_valid = np.load('X_valid.npy')
    y_valid = np.load('y_valid.npy')

    trainset = list(range(len(X_train)))
    valset = list(range(len(X_valid)))
    return trainset, valset, X_train, y_train, X_valid, y_valid

trainset, valset, X_train, y_train, X_valid, y_valid = load_data()
print('trainset = %d samples.' % len(trainset))
print('valset = %d samples.' % len(valset))

trainset = 5969 samples.
valset = 990 samples.


In [2]:
def data_generator(data, params, mode='train'):
    def generator():
        if mode == 'train':
            np.random.shuffle(data)

        for i in data:
            try:
                label_id, mels = (y_train[i], X_train[i]) if mode == 'train' else (y_valid[i], X_valid[i])
                L = 100
                mels_len = mels.shape[1]
                if mels_len < L:
                    print('INVALID DATA | train=%s at %i' % (mode, i))
                    continue
                else:
                    beg = np.random.randint(0, mels_len - L) if mode == 'train' else 0
                yield dict(
                    target=np.int32(label_id),
                    mels=mels[:, beg: beg + L],
                )
            except Exception as err:
                print('data_generator error', err, i)

    return generator

In [3]:
import tensorflow as tf
from tensorflow.contrib import layers

def baseline(x, params, is_training):
    x = layers.batch_norm(x, is_training=is_training)
    for i in range(4):
        x = layers.conv2d(
            x, 16 * (2 ** i), 3, 1,
            activation_fn=tf.nn.elu,
            normalizer_fn=layers.batch_norm if params.use_batch_norm else None,
            normalizer_params={'is_training': is_training}
        )
        x = layers.max_pool2d(x, 2, 2)

    # just take two kind of pooling and then mix them, why not :)
    mpool = tf.reduce_max(x, axis=[1, 2], keep_dims=True)
    apool = tf.reduce_mean(x, axis=[1, 2], keep_dims=True)

    x = 0.5 * (mpool + apool)
    # we can use conv2d 1x1 instead of dense
    x = layers.conv2d(x, 128, 1, 1, activation_fn=tf.nn.elu)
    x = tf.nn.dropout(x, keep_prob=params.keep_prob if is_training else 1.0)
    
    # again conv2d 1x1 instead of dense layer
    logits = layers.conv2d(x, params.num_classes, 1, 1, activation_fn=None)
    return tf.squeeze(logits, [1, 2])

In [4]:
from tensorflow.contrib import signal

# features is a dict with keys: tensors from our datagenerator
# labels also were in features, but excluded in generator_input_fn by target_key

def model_handler(features, labels, mode, params, config):
    # Im really like to use make_template instead of variable_scopes and re-usage
    extractor = tf.make_template(
        'extractor', baseline,
        create_scope_now_=True,
    )
    # mels
    mels = features['mels']
    
    x = tf.stack([mels], axis=3) # shape is [bs, freq_bins, time, 1]
    x = tf.to_float(x)  # we want to have float32, not float64

    logits = extractor(x, params, mode == tf.estimator.ModeKeys.TRAIN)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        # some lr tuner, you could use move interesting functions
        def learning_rate_decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(
                learning_rate, global_step, decay_steps=10000, decay_rate=0.99)

        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer=lambda lr: tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True),
            learning_rate_decay_fn=learning_rate_decay_fn,
            clip_gradients=params.clip_gradients,
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )

    if mode == tf.estimator.ModeKeys.EVAL:
        prediction = tf.argmax(logits, axis=-1)
        acc, acc_op = tf.metrics.mean_per_class_accuracy(
            labels, prediction, params.num_classes)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops=dict(
                acc=(acc, acc_op),
            )
        )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'label': tf.argmax(logits, axis=-1),  # for probability just take tf.nn.softmax()
            'sample': features['sample'], # it's a hack for simplicity
        }
        specs = dict(
            mode=mode,
            predictions=predictions,
        )
    return tf.estimator.EstimatorSpec(**specs)


def create_model(config=None, hparams=None):
    return tf.estimator.Estimator(
        model_fn=model_handler,
        config=config,
        params=hparams,
    )

In [5]:
params=dict(
    seed=2018,
    batch_size=32,
    keep_prob=0.5,
    learning_rate=1e-3,
    clip_gradients=15.0,
    use_batch_norm=True,
    num_classes=len(labels),
)

hparams = tf.contrib.training.HParams(**params)
os.makedirs(os.path.join(OUTDIR, 'eval'), exist_ok=True)
model_dir = OUTDIR

run_config = tf.contrib.learn.RunConfig(model_dir=model_dir)

In [6]:
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn
            
train_input_fn = generator_input_fn(
    x=data_generator(trainset, hparams, 'train'),
    target_key='target',  # you could leave target_key in features, so labels in model_handler will be empty
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)

val_input_fn = generator_input_fn(
    x=data_generator(valset, hparams, 'val'),
    target_key='target',
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)
            

def _create_my_experiment(run_config, hparams):
    exp = tf.contrib.learn.Experiment(
        estimator=create_model(config=run_config, hparams=hparams),
        train_input_fn=train_input_fn,
        eval_input_fn=val_input_fn,
        train_steps=150 * X_train.shape[0] // params['batch_size'],
        eval_steps=200,  # read source code for steps-epochs ariphmetics
        train_steps_per_iteration=X_train.shape[0] // params['batch_size'],
    )
    return exp

tf.contrib.learn.learn_runner.run(
    experiment_fn=_create_my_experiment,
    run_config=run_config,
    schedule="continuous_train_and_eval",
    hparams=hparams)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa757559780>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './sresult2'}
INFO:tensorflow:Training model for 186 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./sresult2/model.ckpt.
INFO:tensorflow:loss = 3.63607, step = 1
INFO:tensorflow:global_step/sec: 134.991
INFO:tensorflow:loss = 2.184096, step = 101 (0.743 sec)
INFO:tensorflow:Saving checkpoints for 186 into ./sresult2/model.ckpt

INFO:tensorflow:Evaluation [178/200]
INFO:tensorflow:Evaluation [179/200]
INFO:tensorflow:Evaluation [180/200]
INFO:tensorflow:Evaluation [181/200]
INFO:tensorflow:Evaluation [182/200]
INFO:tensorflow:Evaluation [183/200]
INFO:tensorflow:Evaluation [184/200]
INFO:tensorflow:Evaluation [185/200]
INFO:tensorflow:Evaluation [186/200]
INFO:tensorflow:Evaluation [187/200]
INFO:tensorflow:Evaluation [188/200]
INFO:tensorflow:Evaluation [189/200]
INFO:tensorflow:Evaluation [190/200]
INFO:tensorflow:Evaluation [191/200]
INFO:tensorflow:Evaluation [192/200]
INFO:tensorflow:Evaluation [193/200]
INFO:tensorflow:Evaluation [194/200]
INFO:tensorflow:Evaluation [195/200]
INFO:tensorflow:Evaluation [196/200]
INFO:tensorflow:Evaluation [197/200]
INFO:tensorflow:Evaluation [198/200]
INFO:tensorflow:Evaluation [199/200]
INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at 2018-03-13-09:20:15
INFO:tensorflow:Saving dict for global step 186: acc = 0.17084233, global_step = 186, loss

INFO:tensorflow:Evaluation [180/200]
INFO:tensorflow:Evaluation [181/200]
INFO:tensorflow:Evaluation [182/200]
INFO:tensorflow:Evaluation [183/200]
INFO:tensorflow:Evaluation [184/200]
INFO:tensorflow:Evaluation [185/200]
INFO:tensorflow:Evaluation [186/200]
INFO:tensorflow:Evaluation [187/200]
INFO:tensorflow:Evaluation [188/200]
INFO:tensorflow:Evaluation [189/200]
INFO:tensorflow:Evaluation [190/200]
INFO:tensorflow:Evaluation [191/200]
INFO:tensorflow:Evaluation [192/200]
INFO:tensorflow:Evaluation [193/200]
INFO:tensorflow:Evaluation [194/200]
INFO:tensorflow:Evaluation [195/200]
INFO:tensorflow:Evaluation [196/200]
INFO:tensorflow:Evaluation [197/200]
INFO:tensorflow:Evaluation [198/200]
INFO:tensorflow:Evaluation [199/200]
INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at 2018-03-13-09:20:19
INFO:tensorflow:Saving dict for global step 372: acc = 0.18889031, global_step = 372, loss = 2.141842
INFO:tensorflow:Training model for 186 steps
INFO:tensorflow:C

INFO:tensorflow:Evaluation [183/200]
INFO:tensorflow:Evaluation [184/200]
INFO:tensorflow:Evaluation [185/200]
INFO:tensorflow:Evaluation [186/200]
INFO:tensorflow:Evaluation [187/200]
INFO:tensorflow:Evaluation [188/200]
INFO:tensorflow:Evaluation [189/200]
INFO:tensorflow:Evaluation [190/200]
INFO:tensorflow:Evaluation [191/200]
INFO:tensorflow:Evaluation [192/200]
INFO:tensorflow:Evaluation [193/200]
INFO:tensorflow:Evaluation [194/200]
INFO:tensorflow:Evaluation [195/200]
INFO:tensorflow:Evaluation [196/200]
INFO:tensorflow:Evaluation [197/200]
INFO:tensorflow:Evaluation [198/200]
INFO:tensorflow:Evaluation [199/200]
INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at 2018-03-13-09:20:23
INFO:tensorflow:Saving dict for global step 558: acc = 0.21656987, global_step = 558, loss = 2.053312
INFO:tensorflow:Training model for 186 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./sresult2/model.ckpt-558
INFO:tensorflow:

INFO:tensorflow:Evaluation [186/200]
INFO:tensorflow:Evaluation [187/200]
INFO:tensorflow:Evaluation [188/200]
INFO:tensorflow:Evaluation [189/200]
INFO:tensorflow:Evaluation [190/200]
INFO:tensorflow:Evaluation [191/200]
INFO:tensorflow:Evaluation [192/200]
INFO:tensorflow:Evaluation [193/200]
INFO:tensorflow:Evaluation [194/200]
INFO:tensorflow:Evaluation [195/200]
INFO:tensorflow:Evaluation [196/200]
INFO:tensorflow:Evaluation [197/200]
INFO:tensorflow:Evaluation [198/200]
INFO:tensorflow:Evaluation [199/200]
INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at 2018-03-13-09:20:27
INFO:tensorflow:Saving dict for global step 744: acc = 0.20155182, global_step = 744, loss = 2.0822256
INFO:tensorflow:Training model for 186 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./sresult2/model.ckpt-744
INFO:tensorflow:Saving checkpoints for 745 into ./sresult2/model.ckpt.
INFO:tensorflow:loss = 0.7943193, step = 745
INFO:tenso

...

INFO:tensorflow:Evaluation [70/200]
INFO:tensorflow:Evaluation [71/200]
INFO:tensorflow:Evaluation [72/200]
INFO:tensorflow:Evaluation [73/200]
INFO:tensorflow:Evaluation [74/200]
INFO:tensorflow:Evaluation [75/200]
INFO:tensorflow:Evaluation [76/200]
INFO:tensorflow:Evaluation [77/200]
INFO:tensorflow:Evaluation [78/200]
INFO:tensorflow:Evaluation [79/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [81/200]
INFO:tensorflow:Evaluation [82/200]
INFO:tensorflow:Evaluation [83/200]
INFO:tensorflow:Evaluation [84/200]
INFO:tensorflow:Evaluation [85/200]
INFO:tensorflow:Evaluation [86/200]
INFO:tensorflow:Evaluation [87/200]
INFO:tensorflow:Evaluation [88/200]
INFO:tensorflow:Evaluation [89/200]
INFO:tensorflow:Evaluation [90/200]
INFO:tensorflow:Evaluation [91/200]
INFO:tensorflow:Evaluation [92/200]
INFO:tensorflow:Evaluation [93/200]
INFO:tensorflow:Evaluation [94/200]
INFO:tensorflow:Evaluation [95/200]
INFO:tensorflow:Evaluation [96/200]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [72/200]
INFO:tensorflow:Evaluation [73/200]
INFO:tensorflow:Evaluation [74/200]
INFO:tensorflow:Evaluation [75/200]
INFO:tensorflow:Evaluation [76/200]
INFO:tensorflow:Evaluation [77/200]
INFO:tensorflow:Evaluation [78/200]
INFO:tensorflow:Evaluation [79/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [81/200]
INFO:tensorflow:Evaluation [82/200]
INFO:tensorflow:Evaluation [83/200]
INFO:tensorflow:Evaluation [84/200]
INFO:tensorflow:Evaluation [85/200]
INFO:tensorflow:Evaluation [86/200]
INFO:tensorflow:Evaluation [87/200]
INFO:tensorflow:Evaluation [88/200]
INFO:tensorflow:Evaluation [89/200]
INFO:tensorflow:Evaluation [90/200]
INFO:tensorflow:Evaluation [91/200]
INFO:tensorflow:Evaluation [92/200]
INFO:tensorflow:Evaluation [93/200]
INFO:tensorflow:Evaluation [94/200]
INFO:tensorflow:Evaluation [95/200]
INFO:tensorflow:Evaluation [96/200]
INFO:tensorflow:Evaluation [97/200]
INFO:tensorflow:Evaluation [98/200]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [74/200]
INFO:tensorflow:Evaluation [75/200]
INFO:tensorflow:Evaluation [76/200]
INFO:tensorflow:Evaluation [77/200]
INFO:tensorflow:Evaluation [78/200]
INFO:tensorflow:Evaluation [79/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [81/200]
INFO:tensorflow:Evaluation [82/200]
INFO:tensorflow:Evaluation [83/200]
INFO:tensorflow:Evaluation [84/200]
INFO:tensorflow:Evaluation [85/200]
INFO:tensorflow:Evaluation [86/200]
INFO:tensorflow:Evaluation [87/200]
INFO:tensorflow:Evaluation [88/200]
INFO:tensorflow:Evaluation [89/200]
INFO:tensorflow:Evaluation [90/200]
INFO:tensorflow:Evaluation [91/200]
INFO:tensorflow:Evaluation [92/200]
INFO:tensorflow:Evaluation [93/200]
INFO:tensorflow:Evaluation [94/200]
INFO:tensorflow:Evaluation [95/200]
INFO:tensorflow:Evaluation [96/200]
INFO:tensorflow:Evaluation [97/200]
INFO:tensorflow:Evaluation [98/200]
INFO:tensorflow:Evaluation [99/200]
INFO:tensorflow:Evaluation [100/200]
INFO:tensorflow:Evaluation 

INFO:tensorflow:Evaluation [76/200]
INFO:tensorflow:Evaluation [77/200]
INFO:tensorflow:Evaluation [78/200]
INFO:tensorflow:Evaluation [79/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [81/200]
INFO:tensorflow:Evaluation [82/200]
INFO:tensorflow:Evaluation [83/200]
INFO:tensorflow:Evaluation [84/200]
INFO:tensorflow:Evaluation [85/200]
INFO:tensorflow:Evaluation [86/200]
INFO:tensorflow:Evaluation [87/200]
INFO:tensorflow:Evaluation [88/200]
INFO:tensorflow:Evaluation [89/200]
INFO:tensorflow:Evaluation [90/200]
INFO:tensorflow:Evaluation [91/200]
INFO:tensorflow:Evaluation [92/200]
INFO:tensorflow:Evaluation [93/200]
INFO:tensorflow:Evaluation [94/200]
INFO:tensorflow:Evaluation [95/200]
INFO:tensorflow:Evaluation [96/200]
INFO:tensorflow:Evaluation [97/200]
INFO:tensorflow:Evaluation [98/200]
INFO:tensorflow:Evaluation [99/200]
INFO:tensorflow:Evaluation [100/200]
INFO:tensorflow:Evaluation [101/200]
INFO:tensorflow:Evaluation [102/200]
INFO:tensorflow:Evaluatio

({'acc': 0.56455857, 'global_step': 28086, 'loss': 1.8611166}, [])